## Setting up and Running FabSim3 in Jupyter Notebook

### 1- Clonning FabSim3 from GitHub

In [ ]:
%%bash

if [ ! -d "FabSim3" ]
then
    git clone https://github.com/djgroen/FabSim3
    echo "FabSim3 rep cloned in -> " $PWD
else
    git -C FabSim3 pull
    echo "Updating FabSim3 repository cloned in -> " $PWD
fi

### 2- Creating and adding SSH Key to authorized keys for SSH authentication

In [ ]:
%%bash

if [ ! -f ~/.ssh/id_rsa ]; then
    ssh-keygen -t rsa -N "" -f ~/.ssh/id_rsa
else
    echo "SSH key already exists"
fi
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
chmod og-wx ~/.ssh/authorized_keys
ssh-keyscan -H localhost >> ~/.ssh/known_hosts

### 3- Installing Python Dependencies for FabSim3

In [ ]:
pip install -r FabSim3/requirements.txt

### 4- Configuring machine_user.yml file specific to a user (e.g., username and directories)

In [ ]:
%%bash

PORT=22

# Configure machines_user.yml
cp FabSim3/fabsim/deploy/machines_user_example.yml FabSim3/fabsim/deploy/machines_user.yml

# Defines your username from local configuration
sed -i "s/your-username/`whoami`/g" FabSim3/fabsim/deploy/machines_user.yml

# Defines port number for localhost connection
sed -i "s#localhost:#localhost:\n  port: $PORT#g" FabSim3/fabsim/deploy/machines_user.yml

# Defines working directory for configs and results directories
sed -i "s#localhost:#localhost:\n  local_configs: \"$PWD/FabSim3/config_files\"#g" FabSim3/fabsim/deploy/machines_user.yml
sed -i "s#localhost:#localhost:\n  local_results: \"$PWD/FabSim3/results\"#g" FabSim3/fabsim/deploy/machines_user.yml

# Defines home path for execution
sed -i "s#localhost:#localhost:\n  home_path_template: \"$PWD/FabSim3/localhost_exe\"#g" FabSim3/fabsim/deploy/machines_user.yml

### 5- Adding FabSim3 to the System PATH and PYTHONPATH

In [ ]:
import os
import sys

# print(os.environ['PATH'])
sys.path.insert(0,  r'%s/FabSim3/fabsim/bin' %(os.getcwd()))
sys.path.insert(0,  r'%s/FabSim3/fabsim' %(os.getcwd()))
os.environ['PATH']=r'%s/FabSim3/fabsim/bin' %(os.getcwd())+os.pathsep+os.environ['PATH']

# Test if fabsim command is availble
!which fabsim

## Installing and Testing FabSim3 with Simple "FabDummy" Plugin, locally

### 1- Installig FabDummy Plugin

In [ ]:
%%bash

# Install FabDummy
fabsim localhost install_plugin:FabDummy

### 2- Running FabDummy test

In [ ]:
%%bash

# Execute a dummy test job
fabsim localhost dummy:dummy_test

### 3- Fetching the Results

In [ ]:
%%bash

fabsim localhost fetch_results

### 4- Checking the Results Directory

In [ ]:
ls $PWD/FabSim3/results/dummy_test_localhost_4

## List All FabSim3 Plugins

In [ ]:
fabsim localhost -l machines

## Cloning and Running Flee using FabFlee Plugin, locally

### 1- Installing FabFlee Plugin

In [ ]:
%%bash

# Install FabFlee 
fabsim localhost install_plugin:FabFlee

### 2- Copying machines_FabFlee_user.yml to machines_FabFlee_user.yml

In [ ]:
%%bash

PORT=22

# Configure machines_FabFlee_user.yml
cp FabSim3/plugins/FabFlee/machines_FabFlee_user_example.yml FabSim3/plugins/FabFlee/machines_FabFlee_user.yml

### 3- Cloning Flee
To run flee locally, we need to install FabFlee Plugin, clone flee locally, and let FabFlee know where flee can be found. We can either add flee_location to machines_user.yml or machines_FabFlee_user.yml

In [ ]:
%%bash

git clone https://github.com/djgroen/flee.git

### 4- Adding flee location to machines_FabFlee_user.yml

In [ ]:
!sed -i  "s#<PATH_TO_FLEE>#$PWD/flee#g" FabSim3/plugins/FabFlee/machines_FabFlee_user.yml

### 5- Running flee locally

In [ ]:
%%bash

fabsim localhost pflee:mali2022_2024,simulation_period=1

## Modifying machine_user.yml file specific to ARCHER2. 
Please set these variables before running the code. The example of ARCHER2 username is "mzr123", project is "e723", and budget is "e723-brunel".

In [ ]:
ARCHER2_username = "ARCHER2 USERNAME"
project_code = "ARCHER2 PROJECT CODE"
budget_code = "ARCHER2 BUDGET CODE"

#### Modifying machine_user.yml file specific to ARCHER2. 

In [ ]:
import yaml

# Path to the machines_user.yml file
machines_user_path = os.path.join("FabSim3", "fabsim", "deploy", "machines_user.yml")

# Load the existing data
with open(machines_user_path, 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

# Update the archer2 section
config['archer2'] = {
    'username': f"{ARCHER2_username}",
    'manual_ssh': True,
    'flee_location': f"/work/{project_code}/{project_code}/{ARCHER2_username}/flee",
    'remote': "archer2",
    'project': f"{project_code}",
    'budget': f"{budget_code}",
    'job_wall_time': "10:00:00",
    'run_prefix_commands': [f"export PYTHONUSERBASE=/work/{project_code}/{project_code}/{ARCHER2_username}/.local", 
                            "export PATH=$PYTHONUSERBASE/bin:$PATH", 
                            "export PYTHONPATH=$PYTHONUSERBASE/lib/python3.8/site-packages:$PYTHONPATH"]
}

# Write the updated data back to the file
with open(machines_user_path, 'w', encoding='utf-8') as file:
    yaml.safe_dump(config, file, default_flow_style=False, sort_keys=False)

#### Modifying machine_FabFlee_user.yml file specific to ARCHER2. 

In [ ]:
import yaml

# Path to the machines_user.yml file
machines_user_path = os.path.join("FabSim3", "plugins", "FabFlee", "machines_FabFlee_user.yml")

# Load the existing data
with open(machines_user_path, 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

# Update the archer2 section
config['archer2'] = {
    'username': f"{ARCHER2_username}",
    'manual_ssh': True,
    'flee_location': f"/work/{project_code}/{project_code}/{ARCHER2_username}/flee",
    'remote': "archer2",
    'project': f"{project_code}",
    'budget': f"{budget_code}",
    'job_wall_time': "10:00:00",
    'run_prefix_commands': [f"export PYTHONUSERBASE=/work/{project_code}/{project_code}/{ARCHER2_username}/.local", 
                            "export PATH=$PYTHONUSERBASE/bin:$PATH", 
                            "export PYTHONPATH=$PYTHONUSERBASE/lib/python3.8/site-packages:$PYTHONPATH"]
}

# Write the updated data back to the file
with open(machines_user_path, 'w', encoding='utf-8') as file:
    yaml.safe_dump(config, file, default_flow_style=False, sort_keys=False)

### Cloning Flee in ARCHER2, Remotely
Same here, to run flee remotley on ARCHER2, we need to clone flee in ARCHER2. We can do this manually by logging to ARCHER2 and clone flee or use the cell below:

In [ ]:
import subprocess

# Clean up the username by stripping any unwanted characters
ARCHER2_username = ARCHER2_username.strip()

# Construct the SSH command
ssh_command = f'ssh {ARCHER2_username}@login.archer2.ac.uk "cd /work/{project_code}/{project_code}/{ARCHER2_username} && if [ ! -d flee ]; then git clone https://github.com/djgroen/flee.git; else echo \"Repository already exists. Skipping clone.\"; fi"'

# Execute the command
try:
    result = subprocess.run(ssh_command, shell=True, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
except Exception as e:
    print(f"SSH command failed: {str(e)}")

### SSH Connection for Running Flee Remotely
To make authentication straightforward, please ssh to archer2 before executing jobs remotely. You can do this in a termainal by issuing command "ssh <ARCHER2_username>@login.archer2.ac.uk"

### Running flee on ARCHER2

In [ ]:
%%bash

fabsim archer2 pflee:mali2022_2024,cores=4,simulation_period=1

### Checking the status of the job

In [ ]:
%%bash 

fabsim archer2 stat

### Fetching the Result 

In [ ]:
%%bash

fabsim archer2 fetch_results

### Experiment with Other Plugins

In [ ]:
%%bash

fabsim localhost avail_plugin

In [ ]:
%%bash

fabsim localhost install_plugin:<plugin from the list of FabSim3 plugins>